In [23]:
import pandas as pd
import numpy as np
from model_settings import ms
from historical_av_key_collector import keys_df, symbol
from historical_av_underlying_fetcher import historical_spots

keys_df = keys_df[keys_df['surface_key'].isna()].dropna(subset=['raw_data_key']).iloc[:1]

contracts = {}


for i,row in keys_df.iterrows():
    with pd.HDFStore(f"alphaVantage {symbol}.h5") as store:
        key = row['raw_data_key']
        date = key[key.find('_',0)+1:key.find("/",1)].replace("_","-")
        contracts[date] = store[row['raw_data_key']]
store.close()

dates = pd.Series(list(contracts.keys()),dtype=str)

In [24]:
historical_spots

,spot_price
date,
2024-10-11,579.5800
2024-10-10,576.1300
2024-10-09,577.1400
2024-10-08,573.1700
2024-10-07,567.8000
...,...
1999-11-05,137.8750
1999-11-04,136.5312
1999-11-03,135.5000


In [25]:
date = dates[0]
df = contracts[date]
spot = float(historical_spots.loc[date].iloc[0])
print(spot)

579.58


In [26]:
df = df.copy()
df

,contractID,symbol,expiration,strike,type,last,mark,bid,bid_size,ask,ask_size,volume,open_interest,date,implied_volatility,delta,gamma,theta,vega,rho
0,SPY241011C00300000,SPY,2024-10-11,300.00,call,276.08,279.26,278.75,1,279.78,1,17,0,2024-10-11,3.87113,1.00000,0.00000,-0.14488,0.00000,0.00822
1,SPY241011P00300000,SPY,2024-10-11,300.00,put,0.00,0.01,0.00,0,0.01,3520,0,2009,2024-10-11,3.79825,-0.00032,0.00001,-0.24831,0.00036,-0.00001
2,SPY241011C00305000,SPY,2024-10-11,305.00,call,0.00,274.26,273.75,1,274.78,1,0,0,2024-10-11,3.80190,1.00000,0.00000,-0.14730,0.00000,0.00836
3,SPY241011P00305000,SPY,2024-10-11,305.00,put,0.00,0.01,0.00,0,0.01,194,0,1211,2024-10-11,3.70755,-0.00033,0.00001,-0.24781,0.00037,-0.00001
4,SPY241011C00310000,SPY,2024-10-11,310.00,call,0.00,269.26,268.75,1,269.78,1,0,0,2024-10-11,3.73267,1.00000,0.00000,-0.14971,0.00000,0.00849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9757,SPY270115P00890000,SPY,2027-01-15,890.00,put,0.00,310.91,308.50,10,313.32,10,0,0,2024-10-11,0.45077,-0.55262,0.00101,-0.03853,3.44800,-14.28438
9758,SPY270115C00895000,SPY,2027-01-15,895.00,call,1.31,1.25,0.75,221,1.74,221,0,2,2024-10-11,0.11327,0.03413,0.00077,-0.02546,0.65989,0.41927
9759,SPY270115P00895000,SPY,2027-01-15,895.00,put,0.00,315.91,313.50,10,318.32,10,0,0,2024-10-11,0.45488,-0.55295,0.00100,-0.03913,3.44762,-14.40113
9760,SPY270115C00900000,SPY,2027-01-15,900.00,call,1.28,1.40,1.15,10,1.66,221,22,498,2024-10-11,0.11709,0.03673,0.00079,-0.02773,0.70087,0.44996


In [28]:
columns_to_convert = ['strike', 'last', 'mark',
       'bid', 'bid_size', 'ask', 'ask_size', 'volume', 'open_interest',
       'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'rho']
df[columns_to_convert] = df[
    columns_to_convert].apply(pd.to_numeric, errors='coerce')

df['expiration'] = pd.to_datetime(df['expiration'],format='%Y-%m-%d')
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')
df['days_to_maturity'] = df['expiration'] - df['date']
df['days_to_maturity'] = df['days_to_maturity'] / np.timedelta64(1, 'D')
df['days_to_maturity'] = df['days_to_maturity'].astype('int64')
df = df[(df['days_to_maturity']>=30)&(df['days_to_maturity']<=400)]

df = df[df['volume']>0].copy()
df['spot_price'] = spot

df['moneyness'] = ms.vmoneyness(df['spot_price'],df['strike'],df['type'])
df = df[(df['moneyness']<0)&(df['moneyness']>-0.5)]
indexed = df.copy().set_index(['strike','days_to_maturity'])

T = np.sort(df['days_to_maturity'].unique()).tolist()
K = np.sort(df['strike'].unique()).tolist()
volume_heatmap = pd.DataFrame(
    np.full((len(K), len(T)), np.nan), index=K, columns=T)
for k in K:
    for t in T:
        try:
            volume_heatmap.loc[k,t] = indexed.loc[(k,t),'volume']
        except Exception:
            pass
        
        
hottest_contracts = pd.DataFrame(
    volume_heatmap.unstack().sort_values(
        ascending=False)).head(50).reset_index()
hottest_contracts.columns = ['t','k','volume']
T = np.sort(hottest_contracts['t'].unique()).tolist()
K = np.sort(hottest_contracts['k'].unique()).tolist()

vol_matrix = pd.DataFrame(
    np.full((len(K),len(T)),np.nan),
    index = K,
    columns = T
)
for k in K:
    for t in T:
        try:
            vol_matrix.loc[k,t] = indexed.loc[(k,float(t)),'implied_volatility']
        except Exception:
            pass

vol_matrix = vol_matrix.dropna().copy()
T = vol_matrix.columns.tolist()
K = vol_matrix.index.tolist()

cols_to_map = [
        'contractID', 'symbol', 'expiration', 'type', 'last', 'mark',
        'bid', 'bid_size', 'ask', 'ask_size', 'volume', 'open_interest', 'date',
        'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'rho',
        'spot_price', 'moneyness'
]
for col in cols_to_map:
    for i,row in hottest_contracts.iterrows():
        hottest_contracts.at[i,col] = indexed.loc[(row['k'],row['t']),col]
        
hottest_contracts = hottest_contracts.rename(
    columns={'t':'days_to_maturity','k':'strike_price'}).copy()

vol_matrix

,35,49,70,81,98,161
400.0,0.53705,0.48293,0.43827,0.41723,0.39970,0.35199
450.0,0.41997,0.38019,0.34985,0.33507,0.32333,0.29376
480.0,0.35001,0.32043,0.30016,0.28903,0.28141,0.26175
490.0,0.32729,0.30138,0.28431,0.27470,0.26815,0.25153
500.0,0.30656,0.28339,0.26937,0.26053,0.25504,0.24163
505.0,0.29544,0.27440,0.26190,0.25382,0.24849,0.23660
520.0,0.26510,0.24864,0.24041,0.23385,0.23004,0.22211
525.0,0.25534,0.24071,0.23370,0.22745,0.22410,0.21724
530.0,0.24605,0.23278,0.22684,0.22105,0.21815,0.21251
535.0,0.23720,0.22486,0.22028,0.21495,0.21221,0.20763
